    # Pràctica 2: Recomanador Simple

Nom dels alumnes del grup:

Nil Torrelles
Arturo Latorre


## 1. INTRODUCCIÓ

### 1.1. Abans de començar...

**\+ A més a més de les que ja es troben presents en la 1a cel·la i funcions natives de Python, durant la pràctica, només es podran fer servir les següents llibreries**:

`Pandas, Numpy, Itertools`

**\+ No es poden modificar les definicions de les funcions donades, ni canviar els noms de les variables i paràmetres ja donats**

Això no implica però que els hàgiu de fer servir. És a dir, que la funció tingui un paràmetre anomenat `df` no implica que l'hàgiu de fer servir, si no ho trobeu convenient.

**\+ En les funcions, s'especifica què serà i de quin tipus cada un dels paràmetres, cal respectar-ho**

Per exemple (ho posarà en el pydoc de la funció), `df` sempre serà indicatiu del `Pandas.DataFrame` de les dades.

### 1.2. Dades: puntuacions de pel·licules

La base de dades [movielens-1M](http://www.grouplens.org/node/73) conté 1,000,209 puntuacions de 3.900 pel·lícules fetes l'any 2000 per 6.040 usuaris anònims del recomanador online [MovieLens](http://www.movielens.org/). 

El consum total de tots els usuaris s'hi pot trobar al document "ratings.dat" el format següent:

    UserID::MovieID::Rating::Timestamp

- **UserID** usuari, amb id's entre 1 i 6040 
- **MovieID** pel·licula, amb id's entre 1 i 3952
- **Rating** puntuació, en una escala de 1 a 5 estrelles.
- **Timestamp** representat en segons

Cada usuari té com a mínim 20 puntuacions.

### 1.3. Dades: usuaris



Al fitxer ``users.dat`` hi trobem la informació referent a cadascun dels usuaris en el següent format:

        UserID::Gender::Age::Occupation::Zip-code

- **Gender** ve donat per "M" per home i "F" per dona.
- **Age** està representada de la següent forma:

	*  1:  "Under 18"
	* 18:  "18-24"
	* 25:  "25-34"
	* 35:  "35-44"
	* 45:  "45-49"
	* 50:  "50-55"
	* 56:  "56+"

- **Occupation** es tria entre les següents opcions:

	*  0:  "other" or not specified
	*  1:  "academic/educator"
	*  2:  "artist"
	*  3:  "clerical/admin"
	*  4:  "college/grad student"
	*  5:  "customer service"
	*  6:  "doctor/health care"
	*  7:  "executive/managerial"
	*  8:  "farmer"
	*  9:  "homemaker"
	* 10:  "K-12 student"
	* 11:  "lawyer"
	* 12:  "programmer"
	* 13:  "retired"
	* 14:  "sales/marketing"
	* 15:  "scientist"
	* 16:  "self-employed"
	* 17:  "technician/engineer"
	* 18:  "tradesman/craftsman"
	* 19:  "unemployed"
	* 20:  "writer"

Els usuaris han donat la informació voluntariament. Així doncs, alguns usuaris poden no tenir informació.


### 1.4. Dades: pel·lícules



Al fitxer ``movies.dat`` hi trobem la informació referent a cadascuna de les películes en el següent format:

        MovieID::Title::Genres

- **Titles** són identics als titols de la base de dades IMDB, incloent l'any de llançament.
- **Genres** de les películes estan separats i seleccionats d'entre els següents:

	* Action
	* Adventure
	* Animation
	* Children's
	* Comedy
	* Crime
	* Documentary
	* Drama
	* Fantasy
	* Film-Noir
	* Horror
	* Musical
	* Mystery
	* Romance
	* Sci-Fi
	* Thriller
	* War
	* Western

Algunes películes poden tenir l'ID malament degut a duplicats accidentals.

Les películes s'han entrat manualment, així que poden existir altres inconsistencies. 

## 2. Exploració de les dades

### 2.1. Descarregar i llegir dades

+ Baixa't els fitxers que composen la base de dades i els còpies al teu directori de treball. 

In [1]:
import os
if os.path.isfile("/etc/password.txt") == False:
    os.system('wget -nc http://files.grouplens.org/datasets/movielens/ml-1m.zip')
    os.system('unzip ml-1m.zip')

+ Llegeix les tres taules de la base de dades en tres DataFrames de pandas amb aquest codi:

In [3]:
import math
import numpy as np
import pandas as pd
import datetime
import itertools
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt

In [5]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python', encoding='latin-1')


### 2.2. Inspecció de les taules

In [4]:
users[:10]

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


In [5]:
users[-10:]

,user_id,gender,age,occupation,zip
6030,6031,F,18,0,45123
6031,6032,M,45,7,55108
6032,6033,M,50,13,78232
6033,6034,M,25,14,94117
6034,6035,F,25,1,78734
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060
6039,6040,M,25,6,11106


In [6]:
ratings[-10:]

,user_id,movie_id,rating,timestamp
1000199,6040,2022,5,956716207
1000200,6040,2028,5,956704519
1000201,6040,1080,4,957717322
1000202,6040,1089,4,956704996
1000203,6040,1090,3,956715518
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


In [7]:
ratings[:10]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [8]:
ratings.sort_values('movie_id')[:5]


,user_id,movie_id,rating,timestamp
427702,2599,1,4,973796689
1966,18,1,4,978154768
683688,4089,1,5,965428947
596207,3626,1,4,966594018
465902,2873,1,5,972784317


In [9]:
movies[:5]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
ratings[:5]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### 2.3 **Exemple:** Com extreure informació d'un DataFrame.

Suposa que volem calcular les **puntuacions mitjanes d'una pel·licula per sexe o edat**, dades que estan a frames diferents.

El primer pas a obtenir una única estructura que contingui tota la informació. Per fer-ho podem usar la funció ``merge`` de pandas. Aquesta funció infereix automàticament quines columnes ha d'usar per fer el ``merge`` basant-se en els noms que fan intersecció.

Reviseu aquests conceptes de pandas: https://pandas.pydata.org/docs/user_guide/merging.html

In [6]:
data = pd.merge(pd.merge(ratings, users), movies)

# Visualitzem la taula ordenada per identificar d'usuari
data.sort_values(by='user_id')[:10]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
28501,1,48,5,978824351,F,1,10,48067,Pocahontas (1995),Animation|Children's|Musical|Romance
13819,1,938,4,978301752,F,1,10,48067,Gigi (1958),Musical
51327,1,1207,4,978300719,F,1,10,48067,To Kill a Mockingbird (1962),Drama
31152,1,1721,4,978300055,F,1,10,48067,Titanic (1997),Drama|Romance
37916,1,2762,4,978302091,F,1,10,48067,"Sixth Sense, The (1999)",Thriller
18472,1,2687,3,978824268,F,1,10,48067,Tarzan (1999),Animation|Children's
45685,1,2692,4,978301570,F,1,10,48067,Run Lola Run (Lola rennt) (1998),Action|Crime|Romance
22832,1,720,3,978300760,F,1,10,48067,Wallace & Gromit: The Best of Aardman Animatio...,Animation
32771,1,745,3,978824268,F,1,10,48067,"Close Shave, A (1995)",Animation|Comedy|Thriller


La funció ``iloc`` ens permet obtenir un subconjunt de files i/o columnes indexades per un enter:

In [12]:
data.iloc[3:5]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


Els índexs Booleans ens permeten seleccionar una part de la taula que compleix una condició.

In [13]:
# comptem quin tant per cent de ratings estan fets per una dona

print(data[data['gender']=='F']['rating'].count()/float(data['rating'].count())*100, '%')

24.638850480249626 %


Per obtenir les **puntuacions mitjanes de cada pel·licula agrupada per edat** podem usar el mètode ``pivot_table`` que és una forma de "canviar" la forma de la taula especificant quin valor agregat (mitjançant una funció predefinida) hi volem en funció dels valors de dues columnes.

Reviseu aquests conceptes: 
+ https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html
+ https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html#pandas.DataFrame.pivot_table

In [14]:
mean_ratings = data.pivot_table(values= 'rating', index='title', columns='age', aggfunc='mean')
mean_ratings[:10]

age,1,18,25,35,45,50,56
title,,,,,,,
"$1,000,000 Duck (1971)",NaN,3.000000,3.090909,3.133333,2.000000,2.750000,NaN
'Night Mother (1986),2.000000,4.666667,3.423077,2.904762,3.833333,3.555556,4.333333
'Til There Was You (1997),3.500000,2.500000,2.666667,2.900000,2.333333,2.500000,2.666667
"'burbs, The (1989)",4.500000,3.244444,2.652174,2.818182,2.545455,3.208333,2.666667
...And Justice for All (1979),3.000000,3.428571,3.724138,3.657143,4.100000,3.551724,3.928571
1-900 (1994),NaN,NaN,2.000000,NaN,NaN,NaN,3.000000
10 Things I Hate About You (1999),3.745455,3.415020,3.432950,3.102941,3.258065,3.629630,4.000000
101 Dalmatians (1961),3.514286,3.295082,3.613757,3.826087,3.976744,3.650000,3.190476
101 Dalmatians (1996),3.088235,2.467742,2.928571,3.279570,3.482759,3.400000,3.555556


Per obtenir les **puntuacions mitjanes de cada pel·licula agrupada per sexe**:

In [15]:
mean_ratings = data.pivot_table('rating', index='title',columns='gender', aggfunc='mean')
mean_ratings[:10]

gender,F,M
title,,
"$1,000,000 Duck (1971)",3.375000,2.761905
'Night Mother (1986),3.388889,3.352941
'Til There Was You (1997),2.675676,2.733333
"'burbs, The (1989)",2.793478,2.962085
...And Justice for All (1979),3.828571,3.689024
1-900 (1994),2.000000,3.000000
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215


Si volgéssim fer càlculs només sobre les pel·licules que han rebut **al menys** 250 puntuacions, primer hem de construir una taula amb el nombre d'avaluacions de cada títol. Per fer-ho, agruparem les dades per títol (amb el mètode ``groupby``) i usarem ``size()`` el nombre.

Reviseu aquest concepte: 

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html

El mètode ``groupby`` implenta un o més d'aquests processos:

+ Dividir les dades segons algun criteri.
+ Aplicar una funció a cada grup.
+ Combinar els resultats en una estructura de dades.

In [16]:
ratings_by_title = data.groupby('title').size()
print(ratings_by_title)

title
$1,000,000 Duck (1971)                         37
'Night Mother (1986)                           70
'Til There Was You (1997)                      52
'burbs, The (1989)                            303
...And Justice for All (1979)                 199
                                             ... 
Zed & Two Noughts, A (1985)                    29
Zero Effect (1998)                            301
Zero Kelvin (Kjærlighetens kjøtere) (1995)      2
Zeus and Roxanne (1997)                        23
eXistenZ (1999)                               410
Length: 3706, dtype: int64


Llavors podem crear un índex amb els títols amb més de 250 avaluacions.

In [17]:
active_titles = ratings_by_title.index[ratings_by_title >= 250]
active_titles

Index([''burbs, The (1989)', '10 Things I Hate About You (1999)',
       '101 Dalmatians (1961)', '101 Dalmatians (1996)', '12 Angry Men (1957)',
       '13th Warrior, The (1999)', '2 Days in the Valley (1996)',
       '20,000 Leagues Under the Sea (1954)', '2001: A Space Odyssey (1968)',
       '2010 (1984)',
       ...
       'X-Men (2000)', 'Year of Living Dangerously (1982)',
       'Yellow Submarine (1968)', 'You've Got Mail (1998)',
       'Young Frankenstein (1974)', 'Young Guns (1988)',
       'Young Guns II (1990)', 'Young Sherlock Holmes (1985)',
       'Zero Effect (1998)', 'eXistenZ (1999)'],
      dtype='object', name='title', length=1216)

L'índex de títols que reben al menys 250 puntuacions es pot fer servir per seleccionar les files de ``mean_ratings``: 

In [18]:
mean_ratings = mean_ratings.loc[active_titles]
mean_ratings

gender,F,M
title,,
"'burbs, The (1989)",2.793478,2.962085
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421
...,...,...
Young Guns (1988),3.371795,3.425620
Young Guns II (1990),2.934783,2.904025
Young Sherlock Holmes (1985),3.514706,3.363344


Per veure els films més valorats per les dones, podem ordenar per la columna F de forma descendent:

In [19]:
top_female_ratings = mean_ratings.sort_values(by='F', ascending=False)
top_female_ratings[:10]

gender,F,M
title,,
"Close Shave, A (1995)",4.644444,4.473795
"Wrong Trousers, The (1993)",4.588235,4.478261
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650,4.464589
Wallace & Gromit: The Best of Aardman Animation (1996),4.563107,4.385075
Schindler's List (1993),4.562602,4.491415
"Shawshank Redemption, The (1994)",4.539075,4.560625
"Grand Day Out, A (1992)",4.537879,4.293255
To Kill a Mockingbird (1962),4.536667,4.372611
Creature Comforts (1990),4.513889,4.272277


Suposem ara que volem les pel·licules que estan valorades de forma més diferent entre homes i dones. Una forma d'obtenir-ho és afegir una columna a ``mean_ratings`` que contingui la diferència en mitjana i llavors ordenar:

In [20]:
mean_ratings['diff'] = mean_ratings['M'] - mean_ratings['F']

In [21]:
print(np.nan + 9.0) 

nan


Ordenant per ``diff`` ens dóna les pel·licules ben valorades per les dones que presenten més diferència entre homes i dones:

In [22]:
sorted_by_diff = mean_ratings.sort_values(by='diff')
sorted_by_diff[:15]

gender,F,M,diff
title,,,
Dirty Dancing (1987),3.790378,2.959596,-0.830782
Jumpin' Jack Flash (1986),3.254717,2.578358,-0.676359
Grease (1978),3.975265,3.367041,-0.608224
Little Women (1994),3.870588,3.321739,-0.548849
Steel Magnolias (1989),3.901734,3.365957,-0.535777
Anastasia (1997),3.800000,3.281609,-0.518391
"Rocky Horror Picture Show, The (1975)",3.673016,3.160131,-0.512885
"Color Purple, The (1985)",4.158192,3.659341,-0.498851
"Age of Innocence, The (1993)",3.827068,3.339506,-0.487561


Invertint l'ordre de les files i fent un ``slicing`` de les 15 files superiors obtenim les pel·licules ben valorades pels homes que no han agradat a les dones: 

In [23]:
sorted_by_diff[::-1][:15]

gender,F,M,diff
title,,,
"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351
"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359
Dumb & Dumber (1994),2.697987,3.336595,0.638608
"Longest Day, The (1962)",3.411765,4.031447,0.619682
"Cable Guy, The (1996)",2.250000,2.863787,0.613787
Evil Dead II (Dead By Dawn) (1987),3.297297,3.909283,0.611985
"Hidden, The (1987)",3.137931,3.745098,0.607167
Rocky III (1982),2.361702,2.943503,0.581801
Caddyshack (1980),3.396135,3.969737,0.573602


Si volguéssim les pel·licules que han generat puntuacions més discordants, independentment del gènere, podem fer servir la variança o la desviació estàndard de les puntuacions: 

In [24]:
# Standard deviation of rating grouped by title
rating_std_by_title = data.groupby('title')['rating'].std()
# Filter down to active_titles
rating_std_by_title = rating_std_by_title.loc[active_titles]
rating_std_by_title.sort_values(ascending=False)[:10]

title
Dumb & Dumber (1994)                     1.321333
Blair Witch Project, The (1999)          1.316368
Natural Born Killers (1994)              1.307198
Tank Girl (1995)                         1.277695
Rocky Horror Picture Show, The (1975)    1.260177
Eyes Wide Shut (1999)                    1.259624
Evita (1996)                             1.253631
Billy Madison (1995)                     1.249970
Fear and Loathing in Las Vegas (1998)    1.246408
Bicentennial Man (1999)                  1.245533
Name: rating, dtype: float64

### Important: Temes de rendiment

Fixeu-vos en el comportament de Python i actueu en conseqüència:

In [25]:
%timeit data['title'] 
print(type(data['title']))
%timeit data.title 
print(type(data.title))
%timeit data[['title']] 
print(type(data[['title']]))

1.78 µs ± 20.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
<class 'pandas.core.series.Series'>
4.03 µs ± 65.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
<class 'pandas.core.series.Series'>
8.01 ms ± 380 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
<class 'pandas.core.frame.DataFrame'>


## 3. EXERCICIS

### 3.1. EXERCICI A

+ Donada la taula ``data`` tal i com es defineix a continuació, calcula la puntuació mitjana de cada usuari i guarda-la a un ``df`` anomenat ``users_mean_rating``. 

In [7]:
data_folder = 'ml-1m'

In [8]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table(f'{data_folder}/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table(f'{data_folder}/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table(f'{data_folder}/movies.dat', sep='::', header=None, names=mnames, engine='python',encoding='latin-1')

data = pd.merge(pd.merge(ratings, users), movies)

#Fem el dataFrame amb el user_id y el rating, utilitzant el mean
users_mean_rating = data.pivot_table(values='rating', index='user_id', aggfunc='mean')

#Imprimim els 15 primers
users_mean_rating[:15]

,rating
user_id,
1,4.188679
2,3.713178
3,3.901961
4,4.190476
5,3.146465
6,3.901408
7,4.322581
8,3.884892
9,3.735849


+ Quina és la pel·lícula més ben puntuada (en mitja) pels usuaris? (Guarda aquest valor en una variable de tipus ``string`` anomenada ``best_movie_rating`` ).

In [28]:
#Fem un df amb els ratings i els titols de les pelicules (es podria fer una trucada directament, pero com reciclarem aquest df en els seguents exercicis, em separat, i em fet el dataframe -> data.pivot_table(values='rating', index='title', aggfunc='mean')['rating'].idxmax()
best_ratings = data.pivot_table(values='rating', index='title', aggfunc='mean')

# Imprimim el titol amb el rating de valor mes alt (idxmean = title)
best_movie_rating = best_ratings['rating'].idxmax()

print(best_movie_rating)

Baby, The (1973)


+ Mira si hi ha més pel·licules amb la mateixa puntuació de la més ben puntuada.

In [29]:
#Agafem el millor rating
best_rating = best_ratings['rating'].max()

#Comparem a veure si hi han mes pelicules
best_rated_movies = best_ratings[best_ratings['rating'] == best_rating]

#Imprimim
best_rated_movies

,rating
title,
"Baby, The (1973)",5.0
Bittersweet Motel (2000),5.0
Follow the Bitch (1998),5.0
"Gate of Heavenly Peace, The (1995)",5.0
Lured (1947),5.0
One Little Indian (1973),5.0
Schlafes Bruder (Brother of Sleep) (1995),5.0
Smashing Time (1967),5.0
Song of Freedom (1936),5.0


+ Busca ara aquella pel·lícula, d'entre les que tenen 5 com a puntuació mitjana, que hagi rebut més valoracions i guarda-la a una variable anomenada ``best_movie_rating_maxviews``. Així tindrem la pel·licula més ben puntuada per més usuaris.

In [30]:
#Comprobem que aquestes pelicules tinguin un 5:
best_rated_movies_5 = best_ratings[best_ratings['rating'] == 5.0]

# Mirem cuantes valoracions hi han dintre de les que tinguin un 5 de valoracio
best_rated_movies_5_num_ratings = data[data['title'].isin(best_rated_movies_5.index)]['title'].value_counts()

# Agafem la que mes tingui
most_viewed_best_rated_movie = best_rated_movies_5_num_ratings.idxmax()

print(most_viewed_best_rated_movie)

Gate of Heavenly Peace, The (1995)


### 3.2. EXERCICI B

+ Defineix una funció anomenada ``top_movie`` que donat un usuari ens retorni quina és la pel·lícula millor puntuada.


In [31]:
def top_movie(dataFrame,usr):
    # Agafem l'usuari especific passat per parametre
    ratings_by_user = dataFrame[dataFrame['user_id'] == usr]

    #Comprobem que existeixi
    if ratings_by_user.empty:
        return "No se encontraron valoraciones para este usuario."

    # Trobem la informacio de la pelicula segons el rating max
    movie_by_userrating = ratings_by_user['rating'].idxmax()

    #Agafem el titol de la pelicula segons el rating max
    top_rated_movie = ratings_by_user.loc[movie_by_userrating, 'title']

    return top_rated_movie
    
print(top_movie(data,1))

One Flew Over the Cuckoo's Nest (1975)


### 3.3. EXERCICI C

+ Construeix una funció que donat el dataframe ``data`` et retorni un altre dataframe ``df_counts``amb el valor que cada usuari li ha donat a una peli. Això ho farem creant un dataframe on les columnes són els `movie_id`, les files `user_id` i els valors siguin el rating donat.

In [9]:
def build_counts_table(df):
    """
    Retorna un dataframe on les columnes són els `movie_id`, les files `user_id` i els valors
    la valoració que un usuari ha donat a una peli d'un `movie_id`
    
    :param df: DataFrame original 
    :return: DataFrame descrit adalt
    """
    
    # la vostra solució aquí
    #Fem el dataframe comentat anteriorment: files: user_id, columnes: movie_id, values=ratings
    df_counts = df.pivot(index='user_id', columns='movie_id', values='rating')

    # Omplim els valors NA amb 0, ja que no tots els usuaris no han valorat totes les pelicules.
    df_counts = df_counts.fillna(0)

    return df_counts

In [10]:
df_counts = build_counts_table(data)
df_counts

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


+ Fés una funció que donada la taula anterior i dos id's (usuari i peli), extregui el valor donat:

In [34]:
def get_count(df, user_id, movie_id):
    """
    Retorna la valoració que l'usuari 'user_id' ha donat de 'movie_id'
    
    :param df: DataFrame retornat per `build_counts_table`
    :param user_id: ID de l'usuari
    :param movie_id: ID de la peli
    :return: Enter amb la valoració de la peli
    """
    
    # la vostra solució aquí
    if user_id in df.index and movie_id in df.columns:
        return df.loc[user_id, movie_id]
    else:
        return "No existeixen tots dos"
    
    

get_count(df_counts, 2, 314)

0.0

### 3.4. EXERCICI D

In [11]:
data.nunique()

user_id         6040
movie_id        3706
rating             5
timestamp     458455
gender             2
age                7
occupation        21
zip             3439
title           3706
genres           301
dtype: int64

In [12]:
unique_movies = pd.unique(data['movie_id'])
print(unique_movies.max())
unique_user = pd.unique(data['user_id'])
print(unique_user.max())

3952
6040


Si observem el nombre total d'usuaris únics i de pel.licules úniques, podem veure que els id's dels usuaris van de 1 a 6040. Normalment volem índexos que comencin al nombre 0, anant de 0 a 6039. 

+ Explora els índexos de les pel·licules. **Quin problema hi ha amb els indexos de les pel·licules??**

> **Resposta**:
> El problema dels indexos es que van desde 1 fins el numero maxim de cada cas (pelicules i usuaris). I el problema ve en que a l'hora de treballar amb aquestos indexos en els casos de consistencia.


+ Usant la funció `pd.Categorical(*).codes`, re-indexa els id's dels usuaris i de les pelis perquè vagin de 0 a 6039 i de 0 a 3705 respectivament:

In [13]:
data['user_id'] = pd.Categorical(data['user_id']).codes
data['movie_id'] = pd.Categorical(data['movie_id']).codes

print(unique_movies.max())
print(unique_user.max())

3952
6040


+ Per comprovar que tot sigui correcte i guardar correctament la taula **df_counts**, torna a calcular i visualitza ``df_counts``:

In [14]:
df_counts = build_counts_table(data)
df_counts

movie_id,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
user_id,,,,,,,,,,,,,,,,,,,,,
0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 3.5. EXERCICI E



+ Escriu una funció `distEuclid(x,y)`  que implementi la distància Euclidiana entre dos vectors usant funcions de pandas. 

+ Escriu la funció `SimEuclid (DataFrame, U1, U2)` que calculin la semblança entre dos usuaris segons aquesta estructura:

    + Calcular un vector per cada usuari, $U1$ i $U2$, amb les puntuacions dels ítems comuns que han puntuat el dos usuaris.
    + Si no hi ha puntuacions en comú, retornar 0. Sinó, retornar 
    
    $$\frac{1}{(1+distEuclid(U1, U2))}$$

+ Avalueu amb la funció ``%timeit`` quant triguen aquests càlculs per un parell d'usuaris.   

> *Nota: Alguns d'aquests exercicis tenen temps de càlcul de l'ordre de minuts sobre tota la base de dades. Per desenvolupar els algorismes és recomanable treballar amb una versió reduïda de la base de dades.* 

Per implementar aquestes funcions únicament es permet l'ús de les funcions:

* `np.sum`
* `np.sqrt`
* `np.power`
* `np.dot`
* `np.linalg.norm`
* `np.mean`

I s'ha de fer **sense bucles**!

In [15]:
num_movies = data.nunique()['movie_id']

def distEuclid(x, y):
    """
    Retorna la distancia euclidiana de dos vectors n-dimensionals.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la distancia euclidiana
    """
    
    return np.sqrt(np.sum(np.power(x-y, 2)))

    
    

def SimEuclid(DataFrame, User1, User2):
    """
    Retorna un score que representa la similitud entre user1 i user2 basada en la distancia euclidiana
    
    :param DataFrame: dataframe que conté totes les dades
    :param User1: id user1
    :param User2: id user2
    :return : Escalar (float) corresponent al score
    """
    #Agafem la informacio de cada usuari
    user1_info = DataFrame[DataFrame['user_id'] == User1]
    user2_info = DataFrame[DataFrame['user_id'] == User2]

    #Pelicules no repetides de cada usuari
    unique_movies_user1 = user1_info['movie_id'].unique()
    unique_movies_user2 = user2_info['movie_id'].unique()


    #Pelicules que tinguin en comu els usuaris
    common_movies = DataFrame[DataFrame['movie_id'].isin(unique_movies_user1) & DataFrame['movie_id'].isin(unique_movies_user2)]


    #Mostrem les ids de cada pelicula en comu
    common_movies_id = common_movies['movie_id'].unique()

    if len(common_movies_id) == 0:
        return 0
    else:
        #Mirem els ratings de cada user de les pelicules que estiguin a les pelicules en comu
        U1_ratings = user1_info[user1_info['movie_id'].isin(common_movies_id)]['rating'].values
        U2_ratings = user2_info[user2_info['movie_id'].isin(common_movies_id)]['rating'].values

        distance = distEuclid(U1_ratings, U2_ratings)

        return (1 / (1 + distance)) * ((len(common_movies_id)/num_movies))



In [16]:
print(SimEuclid(data, 2,314))

0.0005396654074473826


In [17]:
# Execute functions
print(SimEuclid(data, 2,4447))

0.0016759799892629866


In [42]:
# Execute functions
print(SimEuclid(data, 2,1646))

0.0016575437514455324


In [43]:
# Execute functions
print(SimEuclid(data, 2, 4276))

0.001514879601095694


In [44]:
%timeit SimEuclid(data, 1, 5)

16.4 ms ± 543 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 3.6. EXERCICI F

En aquest exercici desenvoluparem un sistema de recomanació col·laboratiu **basat en usuaris**. 

La funció principal, ``getRecommendationsUser``, ha de tenir com a entrada una taula de puntuacions, un ``user_id``, el tipus de mesura de semblança (Euclidiana) que volem usar, el nombre `m` d'usuaris semblants que volem per fer la recomanació i el nombre ``n`` de recomanacions que volem. 

Exemple: ``getRecommendationsUser(data, 2, 50, 10, SimEuclid)``

Com a sortida ha de donar la llista de les $n$ millors pel·lícules que li podriem recomanar segons la seva semblança amb altres usuaris.

> *Nota 1: S'ha d'evitar comparar ``user_id`` a ell mateix.*

> *Nota 2: Recordeu que en Python podem passar funcions com a paràmetres d'una funció.*

#### EXERCICI F.1

+ Computa la *score* de similitud del usuari desitjat (userID) respecte tots els altres i retorna un diccionari dels $m$ usuaris més propers i el seu *score*, que seran els que usarem per fer la recomanació. Normalitzeu els *scores* de sortida.

In [45]:
def find_similar_users(DataFrame, userID, m, simfunction):
    """
    Retorna un diccionari de usuaris similars amb les scores corresponents.
    
    :param DataFrame: dataframe que conté totes les dades
    :param userID: usuari respecte al qual fem la recomanació
    :param m: nombre d'usuaris que volem per fer la recomanació
    :param similarity: mesura de similitud
    :return : dictionary
    """

    similar_users = {}

    #Mirem tots els usuaris al DF
    for user in DataFrame['user_id'].unique():
        #Mirem que no sigui el mateix usuari, es a dir, tots els altres
        if user != userID:
            #Agafem la similitud
            similitud = simfunction(DataFrame, userID, user)
            #Afegim al usuari, la similitud calculada
            similar_users[user] = similitud


    # Ordenem els usuaris per tenirlos ordenats els mes propers
    similar_users = {k: v for k, v in sorted(similar_users.items(), key=lambda item: item[1], reverse=True)}

    # Agafem els m usuaris
    similar_users = dict(list(similar_users.items())[:m])

    # Normalitzem els resultats
    max_score = max(similar_users.values())
    similar_users = {k: v / max_score for k, v in similar_users.items()}

    return similar_users

In [46]:
t = datetime.datetime.now()
sim_dict = find_similar_users(data, 2, 10, SimEuclid)
t = datetime.datetime.now()-t
print(str(t))

0:01:39.839985


In [47]:
sim_dict

{4447: 1.0,
 1646: 0.9889997267654959,
 5830: 0.9299428385089771,
 2270: 0.9113610085905766,
 1903: 0.9094350911298675,
 3271: 0.9039209357006153,
 4276: 0.9038769023500475,
 1879: 0.8865182530712682,
 3625: 0.8846049354012387,
 650: 0.8825141736204608}

+ Creieu que el temps de procés és assumible? Quan trigaria si ho fem per tots els usuaris?

> **RESPOSTA**:
> Aquest temps no es assumible, ja que recorrem els usuaris, unes cuantes vegades, i acortem a M usuaris, i si ho fessim per tots els usuaris, tardaria unes cuantes vegades mes.

+ Per solucionar el problema anterior, construeix una matriu de mida $U \times U$ on cada posició $(i,j)$ indiqui la distància entre l'element $i$ i el $j$. Així doncs, si estàs fent un recomanador basat en usuaris, `matriu[2, 3]` contindrà la similitud entre l'usuari 2 i el 3. Calcula quant triga la teva implementació. 

Heu de tenir en compte que:

* Si feu una funció que treballi amb els vectors de cada usuari i faci un doble ``for``, el procés de les dades pot trigar una bona estona.
* Si feu una funció que treballi específicament amb matrius (i no vectors) trigarà molt pocs segons. En aquest link podeu trobar indicacions de com fer-ho: https://jaykmody.com/blog/distance-matrices-with-numpy/



In [48]:
def compute_distance(fixed_arr, var_arr):
    """
    Dadas dos matrices, calcula la distancia entre los subvectores formados
    por los elementos en común utilizando la distancia euclidiana.
    """
    common_movie_indices = np.intersect1d(np.nonzero(fixed_arr), np.nonzero(var_arr))

    if len(common_movie_indices) == 0:
        return 0

    common_movie_ratings_fixed = fixed_arr[common_movie_indices]
    common_movie_ratings_var = var_arr[common_movie_indices]

    total_pelis = len(common_movie_indices) / df_counts.shape[1]

    dist = (1 / (1 +distEuclid(common_movie_ratings_fixed, common_movie_ratings_var))) * total_pelis

    return dist

def similarity_matrix(compute_distance, df_counts):
    """
    Retorna una matriu de mida M x M on cada posició
    indica la similitud entre usuaris (resp. ítems).

    :param df_counts: df amb els valor que cada usuari li ha donat a una peli.
    :return : Matriu numpy de mida M x M amb les similituds.
    """

    arr = np.array(df_counts)
    num_users = arr.shape[0]

    # Inicializa la matriz de similitudes con ceros
    similarity_matrix = np.zeros((num_users, num_users))

    # Calcula la matriz de similitudes
    for i in tqdm(range(num_users)):
        for j in range(i, num_users):
            # Verifica que i y j sean diferentes para evitar calcular la similitud de un usuario consigo mismo
            similarity_matrix[i, j] = compute_distance(arr[i], arr[j])
            similarity_matrix[j, i] = similarity_matrix[i, j]

    return similarity_matrix

In [50]:
t = datetime.datetime.now()
sim = similarity_matrix(compute_distance, df_counts)
t = datetime.datetime.now()-t
print("Temps amb doble for:",str(t))

  0%|          | 0/6040 [00:00<?, ?it/s]

Temps amb doble for: 0:26:03.368601


In [19]:
print(sim[2][314])

NameError: name 'sim' is not defined

In [22]:
def similarity_matrix_nofors(df_counts):
    """
    Retorna una matriu de mida M x M on cada posició
    indica la similitud entre usuaris (resp. ítems).

    :param df_counts: df amb els valor que cada usuari li ha donat a una peli.
    :return : Matriu numpy de mida M x M amb les similituds.
    """

    arr = df_counts.values
    total_movies = arr.shape[1]


    x2 = np.sum(arr**2, axis=1) # shape of (m)
    y2 = np.sum(arr**2, axis=1) # shape of (n)
    xy = np.matmul(arr, arr.T)
    x2 = x2.reshape(-1, 1)
    dists = np.sqrt(x2 - 2*xy + y2)

    # Calcula la cantidad de películas valoradas en común y el total de películas
    mask =  np.where(arr != 0, 1,0)

    # Calcula la matriz de películas comunes utilizando la operación de multiplicación de matrices
    common_movies = np.matmul(mask, mask.T / total_movies)



    # Normaliza la similitud por la cantidad de películas valoradas en común y el total de películas
    similarities = (1 / (1 + dists)) * common_movies

    return similarities

In [23]:
t = datetime.datetime.now()
sim_nofors = similarity_matrix_nofors(df_counts)
t = datetime.datetime.now()-t
print("Temps sense fors:",str(t))

Temps sense fors: 0:00:03.607838


In [24]:
print(sim_nofors[2][314])

1.5671316708514963e-05


In [25]:
print(sim_nofors)

[[1.43011333e-02 3.63236332e-05 3.99160008e-05 ... 0.00000000e+00
  8.10159742e-05 6.26354958e-05]
 [3.63236332e-05 3.48084188e-02 6.54173950e-05 ... 1.72193823e-05
  3.55436244e-05 1.69970264e-04]
 [3.99160008e-05 6.54173950e-05 1.37614679e-02 ... 2.46354581e-05
  4.23101097e-05 7.43023524e-05]
 ...
 [0.00000000e+00 1.72193823e-05 2.46354581e-05 ... 5.39665407e-03
  4.15460809e-05 4.17798553e-05]
 [8.10159742e-05 3.55436244e-05 4.23101097e-05 ... 4.15460809e-05
  3.31894226e-02 1.49033643e-04]
 [6.26354958e-05 1.69970264e-04 7.43023524e-05 ... 4.17798553e-05
  1.49033643e-04 9.20129520e-02]]


+ Ara torna a re-fer la funció ``find_similar_users`` i mira quant triga.

> Recorda que les scores han d'estar normalitzades!

In [71]:
def find_similar_users(DataFrame, sim_mx, userID, m):
    """
    Retorna un diccionari de usuaris similars amb les scores corresponents.

    :param DataFrame: dataframe que conté totes les dades
    :param sim_mx: Matriu de similitud entre usuaris
    :param userID: usuari respecte al qual fem la recomanació
    :param m: nombre d'usuaris que volem per fer la recomanació
    :return : dictionary
    """
    user_index = DataFrame[DataFrame['user_id'] == userID].index.values[0]

    # Obtenemos la fila correspondiente al usuario en la matriz de similitud
    user_similarity_row = sim_mx[user_index, :]

    # Encontramos los índices de los usuarios más similares
    similar_users_indices = np.argsort(user_similarity_row)[::-1][:m]

    # Obtenemos los IDs de los usuarios más similares y sus similitudes
    similar_users = {DataFrame.iloc[idx]['user_id']: user_similarity_row[idx] for idx in similar_users_indices}

    similar_users.pop(userID, None)

    # Normalizamos los resultados si hay similitudes no nulas
    max_score = max(similar_users.values())
    if max_score != 0:
        similar_users = {k: v / max_score for k, v in similar_users.items()}

    return similar_users
    
    

In [72]:
t = datetime.datetime.now()
sim_dict2 = find_similar_users(data, sim_nofors, 2, 10)
t = datetime.datetime.now()-t
print(str(t))

0:00:00


In [73]:
sim_dict2

{736: 1.0,
 4778: 0.923092884645465,
 1687: 0.913561376701208,
 3978: 0.8996945751539748,
 4646: 0.8957203164595066,
 4725: 0.8879369496538626,
 4047: 0.8856179189562963,
 3644: 0.8823436982431868,
 2516: 0.8744972456757497}

#### EXERCICI F.2

+ Computa les recomanacions per un usuari concret a partir dels ratings dels seus $m$ usuaris més propers. Fes primer una funció que retorni la **weighted average list** dels $m$ usuaris més propers. Feu servir la funció anterior que usava la matriu de similituds per anar més ràpid!! (Nota: la **weighted average list** es calcularà agregant els $n$ items més puntuats de cadascun dels m users més semblants al usuari donat).

In [33]:
def weighted_average(DataFrame, user, sim_mx, m):
    """
    df_counts: usuaris_semblants * pelis
    similarity: {user: similarity}
    
    Aquesta funcio calculara el score predit (Score que ha donat l'usuariB * similitud(A,B)) i despres sumara
    tots els valors per cada peli; retornara {peli_id: suma de score predit}
    """
    similar_users = find_similar_users(DataFrame, sim_mx, user, m)

    # Inicializamos un diccionario para almacenar la suma ponderada de scores por película
    weighted_sum = {}

    for similar_user, similarity_score in similar_users.items():
        # Obtenemos las películas y sus ratings del usuario similar
        user_ratings = DataFrame[DataFrame['user_id'] == similar_user][['movie_id', 'rating']]

        # Calculamos el score ponderado para cada película del usuario similar
        user_ratings['weighted_score'] = user_ratings['rating'] * similarity_score

        # Agregamos el score ponderado al diccionario
        for index, row in user_ratings.iterrows():
            movie_id = row['movie_id']
            weighted_score = row['weighted_score']

            if movie_id not in weighted_sum:
                weighted_sum[movie_id] = weighted_score
            else:
                weighted_sum[movie_id] += weighted_score

    return weighted_sum

In [37]:
def getRecommendationsUser(DataFrame, user, sim_mx, n, m):
    """
    Retorna un dataframe de películas con sus scores predichos.

    :param DataFrame: dataframe que contiene todos los datos
    :param user: usuario al cual hacemos la recomendación
    :param sim_mx: matriz de similitud entre usuarios
    :param n: número de películas a recomendar
    :param m: número de usuarios más cercanos a tener en cuenta para las recomendaciones
    :return: pandas dataframe de películas con sus scores predichos
    """
    # Calcula la lista ponderada para los m usuarios más cercanos al usuario dado
    weighted_sum = weighted_average(DataFrame, user, sim_mx, m)

    # Filtra las películas que el usuario ya ha valorado
    user_rated_movies = DataFrame[DataFrame['user_id'] == user]['movie_id'].tolist()

    # Filtra las películas que ya han sido recomendadas
    recommended_movies = DataFrame[DataFrame['user_id'].isin(weighted_sum.keys())]['movie_id'].tolist()

    # Filtra las películas que no han sido valoradas ni recomendadas
    candidate_movies = set(recommended_movies) - set(user_rated_movies)

    # Ordena las películas candidatas según el score ponderado
    # Ordena las películas candidatas según el score ponderado
    sorted_candidates = sorted(candidate_movies, key=lambda x: weighted_sum.get(x, 0), reverse=True)


    # Selecciona las primeras n películas como recomendaciones
    top_n_recommendations = sorted_candidates[:n]

    # Crea un DataFrame con las recomendaciones y sus scores predichos
    recommendations_df = pd.DataFrame({'movie_id': top_n_recommendations,
                                       'predicted_score': [weighted_sum[movie_id] for movie_id in top_n_recommendations]})

    return recommendations_df


In [43]:
t = datetime.datetime.now()
user_prediction = getRecommendationsUser(data, 5, sim, 10, 50)
t = datetime.datetime.now()-t
print(str(t))

0:00:00.881733


In [44]:
user_prediction

,movie_id,predicted_score
0,253,114.064132
1,1106,105.028108
2,1108,95.227450
3,346,94.076750
4,513,92.456305
5,593,91.167271
6,309,89.865961
7,579,89.517817
8,1107,87.804833
9,1025,87.539511


### 3.7. EXERCICI G


A continuació usarem la metrica **Mean Absolute Error (MAE)** per evaluar el nostre sistema. Aquesta mètrica ens permetrà mesurar la diferencia entre dues llistes donat un usuari: 
+ La llista amb els ratings originals d'un usuari donat
+ La llista de les prediccions generades per aquest usuari

#### EXERCICI G.1

+ Treu el 10% dels usuaris i reserva aquests en una variable anomenada ``test_set`` i la resta en una variable anomenada ``train_set``.

In [45]:
from sklearn.model_selection import train_test_split
# la vostra solució aqu

# Divideix el conjunt de dades en entrenament i prova
train_set, test_set = train_test_split(data, test_size=0.1, random_state=42)


assert len(test_set) + len(train_set) == len(data)

+ Què passarà si calculo la matriu de similitud amb ``train_set`` i després intento predir pels usuaris de ``test_set``??

> **Resposta**: El problema radica en que la matriu de similitud es basaria en les dades d'entrenament, els usuaris del cojunt "test_set" podrien tenir interaccions amb pelicules que no estiguin presents al "train_set", el que provocarí que la predicció fos segurament errónea.


#### EXERCICI G.2

+ Selecciona aproximadament el 80% de les interaccions de cada usuari de test i junta-les al ``train_set``. Podem ara podem evaluar el sistema?


> Com la pràctica és molt llarga us donem el codi per un usuari donat i vosaltres només heu de crear la funció que, per cada usuari, afagi el 80% de les intraccions i les unifiqui al dataframe de train.

In [46]:
test_set.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
895536,5754,178,3,958280246,F,35,2,78744,Nadja (1994),Drama
899739,4584,505,3,964321944,M,35,7,30324,Robocop 3 (1993),Sci-Fi|Thriller
55687,1502,2898,4,974762175,M,25,12,92688,Toy Story 2 (1999),Animation|Children's|Comedy
63727,2165,627,4,974614593,M,1,10,99203,Mission: Impossible (1996),Action|Adventure|Mystery
822011,3200,1997,5,968626301,M,45,7,35216,Frenzy (1972),Thriller


In [47]:
# Agafem el 20% de les pelis que ha consumit cada usuari de test 
groupby_count = test_set.groupby('user_id')['movie_id'].count()*0.2
groupby_count

user_id
0        0.8
1        4.0
2        1.4
3        0.4
4        4.8
        ... 
6035    17.4
6036     3.8
6037     0.2
6038     2.4
6039     5.4
Name: movie_id, Length: 5972, dtype: float64

Seleccionem la posició 1 i aquest use_id serà el que usarem pel codi d'exemple (que després haureu de replicar).

In [48]:
groupby_count.reset_index().iloc[1]

user_id     1.0
movie_id    4.0
Name: 1, dtype: float64

In [49]:
n_test_samples = int(groupby_count.reset_index().iloc[1]['movie_id'])
u = groupby_count.reset_index().iloc[1]['user_id']

In [50]:
test_set_user = test_set[test_set['user_id'] == u]
frame_test = test_set_user.sample(n_test_samples)
print("TOTAL SAMPLES OF THE USER: " + str(len(test_set_user)))
print("TOTAL SAMPLES OF THE USER IN TEST SET: " + str(len(frame_test)))

TOTAL SAMPLES OF THE USER: 20
TOTAL SAMPLES OF THE USER IN TEST SET: 4


In [51]:
len(test_set_user.index)

20

In [52]:
frame_train = test_set_user[~test_set_user.index.isin(frame_test.index)]
print("TOTAL SAMPLES OF THE USER IN TRAIN SET: " + str(len(frame_train)))

TOTAL SAMPLES OF THE USER IN TRAIN SET: 16


In [53]:
assert len(frame_train) + len(frame_test) == len(test_set_user)

In [58]:
def add_testdata(traindf, test_set):
    """
    Retorna els N usuaris més similars basat en la correlació de Pearson (no)
    
    :param traindf: dataframe que conté les dades de train
    :param test_set: dataframe que conté les dades de test

    :return : 
        - :param 1st: dataframe que conté les dades de train juntament amb el 80% de test seleccionat
        - :param 2nd: dataframe que conté les dades de test que queden (20% restant)
    """
    
    # la vostra solució aquí
    for user_id, group_count in test_set.groupby('user_id')['movie_id'].count().items():
        n_test_samples = int(group_count * 0.2)
        test_set_user = test_set[test_set['user_id'] == user_id]
        frame_test = test_set_user.sample(n_test_samples)
        frame_train = test_set_user[~test_set_user.index.isin(frame_test.index)]
        traindf = pd.concat([traindf, frame_train])

    return traindf, test_set

In [55]:
train, test = add_testdata(train_set, test_set)

KeyboardInterrupt: 

In [56]:
train

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
647085,5355,2262,4,960595558,M,45,0,78703,"Fly, The (1986)",Horror|Sci-Fi
130254,3893,1106,3,965789474,M,35,16,02139,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
232200,3107,1546,3,969496514,M,50,6,22039,"Wings of the Dove, The (1997)",Drama|Romance|Thriller
61200,1204,1414,5,974845442,F,56,0,46814,Shall We Dance? (Shall We Dansu?) (1996),Comedy
477192,1703,1278,5,974708453,M,25,4,98926,Batman Returns (1992),Action|Adventure|Comedy|Crime
...,...,...,...,...,...,...,...,...,...,...
981419,6039,78,5,957717093,M,25,6,11106,"White Balloon, The (Badkonake Sefid ) (1995)",Drama
302387,6039,1586,2,956704081,M,25,6,11106,My Life in Pink (Ma vie en rose) (1997),Comedy|Drama
1724,6039,1104,4,957716612,M,25,6,11106,One Flew Over the Cuckoo's Nest (1975),Drama
231375,6039,1293,5,956704716,M,25,6,11106,Citizen Ruth (1996),Comedy|Drama


In [57]:
test

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
895536,5754,178,3,958280246,F,35,2,78744,Nadja (1994),Drama
899739,4584,505,3,964321944,M,35,7,30324,Robocop 3 (1993),Sci-Fi|Thriller
55687,1502,2898,4,974762175,M,25,12,92688,Toy Story 2 (1999),Animation|Children's|Comedy
63727,2165,627,4,974614593,M,1,10,99203,Mission: Impossible (1996),Action|Adventure|Mystery
822011,3200,1997,5,968626301,M,45,7,35216,Frenzy (1972),Thriller
...,...,...,...,...,...,...,...,...,...,...
212290,309,1900,3,976478001,F,18,4,31207,"Little Mermaid, The (1989)",Animation|Children's|Comedy|Musical|Romance
760423,5339,989,4,960703369,F,50,3,95032,Everyone Says I Love You (1996),Comedy|Musical|Romance
298611,1014,852,5,975038812,M,35,3,11220,"Maltese Falcon, The (1941)",Film-Noir|Mystery
219489,755,1288,5,975453580,M,35,0,90064,Jaws (1975),Action|Horror


In [59]:
train.shape

(982649, 10)

In [60]:
test.shape

(100021, 10)

In [61]:
data.shape

(1000209, 10)

In [62]:
assert train.shape[0] + test.shape[0] == data.shape[0]

AssertionError: 

#### EXERCICI G.3

+ Fes una funció que serveixi per evaluar el nostre sistema usant la mètrica MAE. 

In [66]:
def evaluateRecommendations(train, test, m,n, sim):
    """
    Retorna l'error generat pel model
    
    :param DataFrame: dataframe que conté totes les dades
    :param userID: usuari respecte al qual fem la recomanació
    :param m: nombre d'usuaris que volem per fer la recomanació
    :param n: nombre de pelis a retornar (no)
    :param sim: matriu de similitud
    :return : Escalar (float) corresponent al MAE
    """
    # la vostra solució
    
    mae_sum = 0

    for user_id in test['user_id'].unique():
        user_test_set = test[test['user_id'] == user_id]


        # Obtenemos las recomendaciones para el usuario
        recommendations = getRecommendationsUser(train, user_id, sim, n, m)

        # Filtramos las recomendaciones para incluir solo las películas presentes en el conjunto de prueba
        recommendations = recommendations[recommendations['movie_id'].isin(user_test_set['movie_id'])]

        # Calculamos el MAE para este usuario
        mae_user = np.abs(user_test_set['rating'] - recommendations['predicted_score']).mean()

        # Agregamos el MAE del usuario al total
        mae_sum += mae_user

    # Calculamos el MAE promedio para todos los usuarios
    mae = mae_sum / len(test['user_id'].unique())
    return mae
    


In [67]:
t = datetime.datetime.now()
mae = evaluateRecommendations(train, test, 50, 10, sim)
t = datetime.datetime.now()-t
print(str(t))

IndexError: index 705342 is out of bounds for axis 0 with size 6040

In [65]:
mae

NameError: name 'mae' is not defined

### 3.8. EXERCICI H (exercici opcional, no obligatori)


+ **Que surt més a compte, fer un recomanador unic pels dos sexes o un per cada sexe?** Justifica la resposta per escrit i amb el codi necessari.